In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pprint as pp

In [22]:
url = "https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos"
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")
for a_href in soup.findAll(class_='summary url'):
    print(a_href["href"])
#link_container = soup.findAll(class_='summary url')
#print(link_container)

https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-sessao-especial-do-centro-global-rei-hamad-para-a-coexistencia-pacifica-manama-bahrein
https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-solenidade-de-lancamento-do-programa-brasil-fraterno-comida-no-prato-palacio-do-planalto
https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-solenidade-de-consolidacao-do-marco-regulatorio-trabalhista-infralegal-palacio-do-planalto
https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-anuncios-do-governo-federal-ao-estado-do-parana
https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-no-leilao-do-5g-brasilia-df
https://www.gov.br/planalto

In [24]:
archive_links = ["https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-sessao-especial-do-centro-global-rei-hamad-para-a-coexistencia-pacifica-manama-bahrein",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-solenidade-de-lancamento-do-programa-brasil-fraterno-comida-no-prato-palacio-do-planalto",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-solenidade-de-consolidacao-do-marco-regulatorio-trabalhista-infralegal-palacio-do-planalto",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-anuncios-do-governo-federal-ao-estado-do-parana",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-no-leilao-do-5g-brasilia-df",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-no-5o-forum-nacional-de-controle-educacao-no-pos-pandemia-desafios-e-oportunidades-brasilia-df",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-em-memoria-dos-pracinhas-brasileiros-mortos-na-segunda-guerra-mundial-pistoia-italia",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/mensagem-do-presidente-da-republica-jair-bolsonaro-na-conferencia-das-nacoes-unidas-sobre-mudancas-climaticas-de-2021",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-sessao-i-economia-e-saude-global-da-16-cupula-do-g20-roma-italia",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-alusiva-ao-dia-do-servidor-publico-palacio-do-planalto",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-primeira-consagracao-publica-de-pastores-do-estado-do-amazonas-manaus-am",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-no-culto-em-comemoracao-aos-106-anos-da-assembleia-de-deus-manaus-am",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-inauguracao-do-ramal-do-agreste-sertania-pe",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-inauguracao-da-obra-do-trecho-final-do-eixo-norte-do-projeto-de-integracao-do-rio-sao-francisco-sao-jose-de-piranhas-pb",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-de-cerimonia-de-sancao-do-pl-5919-19-palacio-do-planalto",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-jornada-das-aguas-lancamento-do-edital-de-construcao-do-ramal-de-salgado-russas-ce",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/declaracao-conjunta-do-presidente-da-republica-jair-bolsonaro-e-do-presidente-da-colombia-senhor-ivan-duque-marquez-apos-visita-oficial-no-palacio-do-planalto",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-alusiva-ao-lancamento-da-jornada-das-aguas-sao-roque-mg",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-conferencia-global-2021-millenium-brasilia-df",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-entrega-de-titulos-de-propriedade-rural-miracatu-sp",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-cerimonia-alusiva-a-1a-feira-brasileira-do-niobio-sirius-2013-hall-experimental-2013-c10-campinas-sp",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-cerimonia-de-modernizacao-de-normas-de-seguranca-e-saude-no-trabalho-palacio-do-planalto",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-recepcao-aos-atletas-olimpicos-e-paralimpicos-palacio-do-planalto",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-entrega-das-obras-de-modernizacao-do-aeroporto-de-maringa-e-de-inauguracao-da-hidreletrica-de-bela-vista-maringa-pr",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-assinatura-do-decreto-da-cedula-de-produto-rural-verde-palacio-do-planalto",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-sancao-do-pln-para-obras-do-metro-de-bh-e-de-lancamento-de-pedra-fundamental-do-centro-nacional-de-vacinas-mcti-2013-ufmg-belo-horizonte-mg",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-no-ato-publico-com-o-governador-de-roraima-antonio-denarium-boa-vista-rr",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-alusiva-a-inauguracao-da-ute-jaguatirica-ii-a-concessao-dos-aeroportos-do-bloco-norte-e-a-entrega-de-veiculos-do-programa-alimenta-brasil-boa-vista-rr",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-entrega-do-residencial-dr-marcelo-vilela-teotonio-vilela-al",
"https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/discursos/2021/discurso-do-presidente-da-republica-jair-bolsonaro-na-cerimonia-de-inauguracao-da-estacao-cidadania-de-entrega-de-titulos-de-propriedades-rurais-e-de-anuncio-de-duplicacao-da-br-116-e-da-br-101-teixeira-de-freitas-ba"]


In [25]:
for link in archive_links:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    
    for a_href in soup.findAll(class_='summary url'):
        with open("link_test.txt","a") as linkfile:
            linkfile.write(a_href["href"]+"\n")

link_test.txt
